In [2]:
import pandas as pd
import altair as alt
import eco_style
import numpy as np

alt.themes.enable('light')

ThemeRegistry.enable('light')

## Fig 1

In [2]:
df

NameError: name 'df' is not defined

In [3]:
df = pd.read_csv("fig1.csv")
df = df.drop(columns=["year"])

df = df.melt(id_vars=["date"], var_name="series", value_name="value")

df.value = df.value / 100

df.date = pd.to_datetime(df.date, format="%d/%m/%Y")

base = alt.Chart(df).encode(
    x=alt.X("date:T"),
    y=alt.Y("value:Q", title="", axis=alt.Axis(format=".0%"), scale=alt.Scale(domain=[-0.08, 0.1])),
    color=alt.Color("series:N", title="", legend=alt.Legend(
        orient="none",
        legendX=10,
        legendY=-15,
        direction="horizontal",
    ))
)

line = base.mark_line()

chart = line

chart.display()

chart.save("fig1.json")
chart.save("fig1.png", scale_factor=3)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


## Fig 2

In [22]:
df = pd.read_excel("EO_Feb24.xlsx", sheet_name="Fig2", skiprows=6)

df = df.iloc[1:]

df = df[[c for c in df.columns if "Unnamed" not in c]]

def period_midpoint(x):
    """
    e.g. "Jan-Mar 2020" -> "2020-02-01"
    """
    year = x.split(" ")[1]
    period_start_month = x.split(" ")[0].split("-")[0]
    period_end_month = x.split(" ")[0].split("-")[1]

    months_indexes = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    period_start_month_index = months_indexes.index(period_start_month)
    period_end_month_index = months_indexes.index(period_end_month)
    mid_month_index = (period_start_month_index + period_end_month_index) // 2

    date = f"{year}-{mid_month_index + 1:02d}-01"
    return pd.to_datetime(date)

def date(x):
    """
    e.g. "Jan-Mar 2020" -> "2020-01-01"
    """
    year = x.split(" ")[1]
    period_start_month = x.split(" ")[0].split("-")[0]

    date = f"{year}-{period_start_month}-01"
    return pd.to_datetime(date)



df['date'] = df['Period'].apply(date)

df = df.melt(id_vars=["date", "Period"], var_name="series", value_name="value")

df['label'] = np.where(df['Period'] == "Oct-Dec 2023", df.series, "")


base = alt.Chart(df).encode(
    x=alt.X("date:T",
            # remove space between bars
            axis=alt.Axis(labelAngle=0, labelPadding=0, tickCount=10, title="", format="%b %Y")
            
            ),
    y=alt.Y("value:Q",
            # stack the bars
            stack="zero",
             title="", axis=alt.Axis()),
    color=alt.Color("series:N", title="", legend=None)
)

bars = base.transform_filter(" datum.series !=  'Economically inactive (16 to 64)'").mark_area(interpolate='step-after').encode(
    tooltip=[
        {"type": "temporal", "field": "Period", "title": "Period"},
        {"type": "quantitative", "field": "value", "title": "Value"},
        {"type": "nominal", "field": "series", "title": "Series"}
    ]
)

# add a line right before July-Sep 2022
rule = alt.Chart(pd.DataFrame([{"date": pd.to_datetime("2022-09-01"), "value": 0}])).mark_rule(color="black").encode(
    x="date:T"
)

reweight_label = alt.Chart(pd.DataFrame([{"date": pd.to_datetime("2022-09-01"), "value": 0.05, "label": "Re-weighted data"}])).mark_text(align='left', dx=7).encode(
    x="date:T",
    y=alt.value(8),
    text="label:N"
)

# the overall line
line = base.transform_filter(" datum.series ==  'Economically inactive (16 to 64)'").mark_line().encode(
    color=alt.value("black"),
)

line_label = base.transform_filter(" datum.series ==  'Economically inactive (16 to 64)'").mark_text(align='left', dx=22, dy=25).encode(
    text=alt.Text("label:N"),
    color=alt.value("black"),
)


labels = base.transform_filter(" datum.series !=  'Economically inactive (16 to 64)'").mark_area(interpolate='step-after').mark_text(align='left',
                        dy=alt.expr("{\
                                        'Long-term sick': 5,\
                                        'Student': 15,\
                                        'Temp sick': -5,\
                                        'Retired': 5,\
                                        'Discouraged workers': 10,\
                                    }[datum.series] \
                                    "),
                         baseline='middle', dx=22).encode(
    text=alt.Text('label:N'),
)


chart = bars + labels + rule + reweight_label + line + line_label


chart = chart.configure_scale(
    bandPaddingInner=0

)

chart.save("fig2.json")
chart.save("fig2.png", scale_factor=3)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [ ]:
chart.

In [15]:
df.query("label != ''")

,date,Period,series,value,label
45,2023-10-01,Oct-Dec 2023,Economically inactive (16 to 64),908413.454251,Economically inactive (16 to 64)
91,2023-10-01,Oct-Dec 2023,Student,371134.549537,Student
137,2023-10-01,Oct-Dec 2023,Looking after family or home,-190607.857204,Looking after family or home
183,2023-10-01,Oct-Dec 2023,Temp sick,-17321.667702,Temp sick
229,2023-10-01,Oct-Dec 2023,Long-term sick,687891.909582,Long-term sick
275,2023-10-01,Oct-Dec 2023,Discouraged workers,-6599.936970,Discouraged workers
321,2023-10-01,Oct-Dec 2023,Retired,-33457.462342,Retired
367,2023-10-01,Oct-Dec 2023,Other,97373.919351,Other


In [7]:
df

,date,Period,series,value,label
0,2020-02-01,Jan-Mar 2020,Economically inactive (16 to 64),76299.905427,
1,2020-03-01,Feb-Apr 2020,Economically inactive (16 to 64),195851.067423,
2,2020-04-01,Mar-May 2020,Economically inactive (16 to 64),224229.552935,
3,2020-05-01,Apr-Jun 2020,Economically inactive (16 to 64),290236.624399,
4,2020-06-01,May-Jul 2020,Economically inactive (16 to 64),283555.070086,
...,...,...,...,...,...
363,2023-07-01,Jun-Aug 2023,Other,139639.006617,
364,2023-08-01,Jul-Sep 2023,Other,133345.499722,
365,2023-09-01,Aug-Oct 2023,Other,111832.321772,
366,2023-10-01,Sep-Nov 2023,Other,156560.704610,


## Fig 3

In [159]:
df = pd.read_excel("EO_Feb24.xlsx", sheet_name="Fig3", skiprows=2)

df = df.iloc[:, 1:]
df = df.rename(columns={"Unnamed: 1": "year"})

df = df[[c for c in df.columns if "Unnamed" not in c]]

df = df.dropna(subset=["year"])

df["date"] = pd.to_datetime(df.year.astype(int).astype(str) + "-" + (df.index % 12 + 1).astype(str) + "-01")

df = df.melt(id_vars=["date", "year"], var_name="series", value_name="value")

df['value'] = df['value'] / 100

df['label'] = np.where(df['date'] == "2024-01-01", df.series, "")

base = alt.Chart(df).encode(
    x=alt.X("date:T", title="", axis=alt.Axis(labelAngle=0, labelPadding=0, tickCount=10, format="%b %Y")),
    y=alt.Y("value:Q", title="", axis=alt.Axis(format=".0%")),
    color=alt.Color("series:N", title="", legend=None)
)

bars = base.transform_filter("datum.series != 'Total UK CPI'").mark_area(interpolate='step-after').encode(
    tooltip=[
        {"type": "temporal", "field": "date", "title": "Date"},
        {"type": "quantitative", "field": "value", "title": "Value"},
        {"type": "nominal", "field": "series", "title": "Series"}
    ]
)

labels = base.transform_filter("datum.series != 'Total UK CPI'").mark_text(align='left',
                                dy=alt.expr("{\
                                        'Food': -87,\
                                        'Services': 15,\
                                        'Non-energy industrial goods': -55,\
                                        'Retired': 5,\
                                        'Discouraged workers': 10,\
                                    }[datum.series] \
                                    "),
                         baseline='middle', dx=3).encode(
    text=alt.Text('label:N'),
)

line = base.transform_filter("datum.series == 'Total UK CPI'").mark_line().encode(
    color=alt.value("black"),
)

line_label = base.transform_filter("datum.series == 'Total UK CPI'").mark_text(align='left', dx=3).encode(
    text=alt.Text('label:N'),
    color=alt.value("black"),
)

chart = bars + labels + line + line_label

chart.save("fig3.json")
chart.save("fig3.png", scale_factor=3)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

## Fig 4

In [167]:
df = pd.read_excel("EO_Feb24.xlsx", sheet_name="Figure 4", skiprows=1)

df = df.iloc[:, :3]

df.columns = ["date", "actual", "forecast"]

df = df.melt(id_vars=["date"], var_name="series", value_name="value")

df = df.dropna(subset=["value"])

base = alt.Chart(df).encode(
    x=alt.X("date:T", title="", axis=alt.Axis(labelAngle=0, labelPadding=0, tickCount=10, format="%b %Y")),
    y=alt.Y("value:Q", title="", axis=alt.Axis()),
    color=alt.Color("series:N", title="", legend=None)
)

line = base.mark_line()

# add a dividing rule at 2023-12-01
rule = alt.Chart(pd.DataFrame([{"date": pd.to_datetime("2023-12-01"), "value": 0}])).mark_rule(color="black").encode(
    x="date:T"
)

# labels
labels = alt.Chart(pd.DataFrame([
    {"date": pd.to_datetime("2023-12-30"), "value": 10, "series": "forecast", "label": "NIESR Forecast →"},
    {"date": pd.to_datetime("2023-04-15"), "value": 10, "series":"actual", "label": "← ONS Data"}
                                 ])).mark_text(align='left', dx=0).encode(
    x="date:T",
    y=alt.value(8),
    text="label:N",
    color="series:N"
)

chart = line + rule + labels

chart = chart.properties(
    width=500
)

chart.save("fig4.json")
chart.save("fig4.png", scale_factor=3)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [132]:
df

,date,series,value
0,2019-12-01,actual,100.000000
1,2020-01-01,actual,100.000000
2,2020-02-01,actual,99.603175
3,2020-03-01,actual,92.063492
4,2020-04-01,actual,74.007937
5,2020-05-01,actual,75.595238
6,2020-06-01,actual,82.936508
7,2020-07-01,actual,88.392857
8,2020-08-01,actual,91.071429
9,2020-09-01,actual,92.261905


In [107]:
df.series.value_counts()

series
Food                           61
Energy                         61
Total UK CPI                   61
Services                       61
Non-energy industrial goods    61
Name: count, dtype: int64

In [98]:
df[df.date == "2024-01-01"]

,date,year,series,value,label
60,2024-01-01,2024.0,Food,0.007790,
121,2024-01-01,2024.0,Energy,-0.010439,
182,2024-01-01,2024.0,Total UK CPI,0.040000,
243,2024-01-01,2024.0,Services,0.031928,
304,2024-01-01,2024.0,Non-energy industrial goods,0.007769,


In [92]:
df

,date,year,series,value
0,2019-01-01,2019.0,Food,0.000885
1,2019-02-01,2019.0,Food,0.001079
2,2019-03-01,2019.0,Food,0.000782
3,2019-04-01,2019.0,Food,0.000686
4,2019-05-01,2019.0,Food,0.000981
...,...,...,...,...
300,2023-09-01,2023.0,Non-energy industrial goods,0.013892
301,2023-10-01,2023.0,Non-energy industrial goods,0.012554
302,2023-11-01,2023.0,Non-energy industrial goods,0.009492
303,2023-12-01,2023.0,Non-energy industrial goods,0.009083


In [85]:
13%12

1

In [41]:
df.series.unique()

array(['Economically inactive (16 to 64)', 'Student',
       'Looking after family or home', 'Temp sick', 'Long-term sick',
       'Discouraged workers', 'Retired', 'Other'], dtype=object)

In [15]:
labels

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [52]:
df.to_csv("fig2_cleaned.csv", index=False)

In [17]:
df

,date,Period,series,value
0,2020-02-01,Jan-Mar 2020,Economically inactive (16 to 64),76299.905427
1,2020-03-01,Feb-Apr 2020,Economically inactive (16 to 64),195851.067423
2,2020-04-01,Mar-May 2020,Economically inactive (16 to 64),224229.552935
3,2020-05-01,Apr-Jun 2020,Economically inactive (16 to 64),290236.624399
4,2020-06-01,May-Jul 2020,Economically inactive (16 to 64),283555.070086
...,...,...,...,...
363,2023-07-01,Jun-Aug 2023,Other,139639.006617
364,2023-08-01,Jul-Sep 2023,Other,133345.499722
365,2023-09-01,Aug-Oct 2023,Other,111832.321772
366,2023-10-01,Sep-Nov 2023,Other,156560.704610


In [15]:
df.columns[1]

datetime.datetime(2016, 3, 1, 0, 0)